In [1]:
#Import required libraries 
import keras #library for neural network
import pandas as pd #loading data in table form  
import seaborn as sns #visualisation 
import matplotlib.pyplot as plt #visualisation
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from sklearn.preprocessing import normalize #machine learning algorithm library
from keras.models import Sequential 
from keras.layers import Dense,Activation,Dropout 
from keras.utils.vis_utils import plot_model
import math

In [2]:
#Reading data 
data=pd.read_csv("IrisData.csv")
col_names=list(data.columns.values)
num_line = len(data.index)

num_input=4
num_output=3
num_var=5
clusters=3
#print("Describing the data: ",data.describe())
#print("Info of the data:",data.info())
data

,SepalLength,SepalWidth,PetalLength,PetalWidth,TYPE
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.4,3.1,5.5,1.8,2
148,6.2,3.4,5.4,2.3,2


In [3]:
#Reading data 
train_num_line=int(num_line*0.2)
test_num_line = num_line - train_num_line
# test_num_line=num_line
# train_num_line=num_line

# Create data_train = initialise dataframe (80%)
temp = np.array([[0.0 for i in range(train_num_line)] for j in range(num_var)]).T
# Array to dataframe :
data_train = pd.DataFrame(temp,columns=col_names)

# Create data_train = initialise dataframe (80%)
temp = np.array([[0.0 for i in range(test_num_line)] for j in range(num_var)]).T
# Array to dataframe :
data_test = pd.DataFrame(temp,columns=col_names)

count = 0
train_count=0
test_count=0

# # 20% train, 80% test
# # 0,1 // 2,3 // 4,5 // 6,7 // 8,9
# for i in range(num_line):
#     if (count==8 or count ==9):
#         data_train.iloc[train_count] = data.iloc[i]
#         train_count=train_count+1
#     else:
#         data_test.iloc[test_count] = data.iloc[i]
#         test_count=test_count+1
#     count=count+1
#     if count==10:
#         count=0

# ## odd even
# for i in range(num_line):
#     #even
#     if (count==0):
#         data_test.iloc[test_count] = data.iloc[i]
#         test_count=test_count+1
#         count=1
#     ##odd
#     else:
#         data_train.iloc[train_count] = data.iloc[i]
#         train_count=train_count+1
#         count=0

data_test=data
data_train=data_test
##odd
print(data_train)
print("train_num_line ",train_num_line)


##even
print(data_test)
print("test_num_line",test_num_line)

     SepalLength  SepalWidth  PetalLength  PetalWidth  TYPE
0            5.1         3.5          1.4         0.2     0
1            4.9         3.0          1.4         0.2     0
2            4.7         3.2          1.3         0.2     0
3            4.6         3.1          1.5         0.2     0
4            5.0         3.6          1.4         0.2     0
..           ...         ...          ...         ...   ...
145          6.7         3.0          5.2         2.3     2
146          6.3         2.5          5.0         1.9     2
147          6.4         3.1          5.5         1.8     2
148          6.2         3.4          5.4         2.3     2
149          5.9         3.0          5.1         1.8     2

[150 rows x 5 columns]
train_num_line  30
     SepalLength  SepalWidth  PetalLength  PetalWidth  TYPE
0            5.1         3.5          1.4         0.2     0
1            4.9         3.0          1.4         0.2     0
2            4.7         3.2          1.3         0.2    

In [4]:
learningConst = float(1/train_num_line)
epsilon = 0.0005
discreteSamplePoints=50
mlvq_lwidth=1.5
pfkp_lwidth=0.2
c=0

In [5]:
class network():
  def __init__(self,dim):
    self.num_layers=dim
    self.layers = [layer]*dim
    self.num_feature=0
    self.num_pattern=0
  def addLayers(self,i,layer):
    self.layers[i]=layer

In [6]:
class layer(network):
  def __init__(self,dim,ipNeuron,x):
    self.dim = dim
    self.neus = [neuron]*dim
    self.ipneus=[ipneuron]*dim
    self.count=0
    for i in range(dim):
        if ipNeuron == 0:
            self.neus[i]=neuron(dim)
        else:
            self.ipneus[i]=ipneuron(dim,i,x)

In [7]:
class neuron(layer):
  def __init__(self,dim):
    self.name = None
    self.value = 0
    self.num_inputs = 0
    self.input = [neuron]*num_input
    self.wt= [0]*num_input
    self.combFunc=-1
    self.actFunc=-1
    self.activation=0
    self.ip_type=1
    self.ipneu_input = [ipneuron]
    self.actFunc_arg=[0]

In [8]:
class ipneuron(layer):
  def __init__(self,dim,count,x):
    self.numPt=discreteSamplePoints
    self.ip_value=0
    self.value=[0]*num_output #num_pattern
    self.name=col_names[count]
    self.mf = [[0 for i in range(discreteSamplePoints)] for j in range(num_output+1)] 
    for i in range(num_output+1):
        for j in range(discreteSamplePoints):
            #print((discreteSamplePoints*i)+j+(count*(discreteSamplePoints*(num_output+1))))
            self.mf[i][j]=x[(discreteSamplePoints*i)+j+(count*(discreteSamplePoints*(num_output+1)))]
    if(count==num_input-1):
        count=count+1
        for i in range(clusters+1):
            for j in range(discreteSamplePoints):
#                 print((discreteSamplePoints*i)+j+(count*(discreteSamplePoints*(output_cluster+1))))
                outmf[i][j]=x[(discreteSamplePoints*i)+j+(count*(discreteSamplePoints*(clusters+1)))]

In [9]:
def buildConnections(source, dest, start, step, size): #layer2->layer3, 0-3
    i=k=st=start
    for i in range(dest.dim):
        currneu=dest.neus[i]
        #newneu=[None]*8
        newneu=[neuron]*8
        newwt=[0]*8
        for k in range(currneu.num_inputs):
            newneu[k] = currneu.input[k]
            newwt[k] = currneu.wt[k]
        currneu.input=newneu
        currneu.wt = newwt
        k=st
        for k in range(st+size):
            currneu.input[currneu.num_inputs]=source.neus[k]
            currneu.wt[currneu.num_inputs]=0
            currneu.num_inputs+=currneu.num_inputs
        st+=step
    return

In [10]:
def findWinner(value,lyr):
    winner=0
    error=math.fabs(value-lyr.neus[0].value)
    for i in range(1,lyr.dim):
        temp=min(error, math.fabs(value-lyr.neus[i].value))
        if temp<error:
            error=temp
            winner=i
    return winner

In [11]:
def findWinner1(value,pt,numPt):
    winner=0
    error=math.fabs(value-pt[0])
    for i in range(1,numPt):
        temp=min(error, math.fabs(value-pt[i]))
        if temp<error:
            error=temp
            winner=i
    return winner

In [12]:
def dct(nw,data,ndx):
    e1=0
    le1=0
    e2=0
    le2=0
    neighbours = 0
    iterations=0
    min1=max1=data[0]
    
    for k in range(train_num_line):
        max1=max(max1,data[k])
        min1=min(min1,data[k])
    for j in range(1,nw.num_layers):
        lyr=nw.layers[j]
        for i in range(lyr.dim):
            lyr.neus[i].value=min1+(i+0.5)/(lyr.dim)*(max1-min1)
    
    neighbours = int(nw.layers[2].dim/5);
    interval = (max1-min1)/nw.layers[2].dim;
    
    for j in range(nw.layers[1].dim): 
        lyr = nw.layers[2]
        for i in range(lyr.dim):
            lyr.neus[i].wt[j]=0
    
    while(1):
        for k in range(train_num_line):
            winner1=findWinner(data[k],nw.layers[1])
            e1+= math.fabs(data[k]-nw.layers[1].neus[winner1].value)
            nw.layers[1].neus[winner1].value *= (1-learningConst)
            nw.layers[1].neus[winner1].value += learningConst*data[k];
            
            
            lyr = nw.layers[2];
            winner2 = findWinner(data[k], lyr);
            diff = math.fabs(data[k]-lyr.neus[winner2].value);
            lyr.neus[winner2].wt[winner1] += learningConst*(1-diff/interval)
            e2 += diff;
            
            
            i=(winner2-1)
            for l in range(2,neighbours+2):
                if i>=0:
                    diff = math.fabs(data[k]-lyr.neus[i].value)
                    lyr.neus[i].wt[winner1] += learningConst/l*(1-diff/(l*interval));
                i=i-1
                
            i=(winner2+1)
            for l in range(2,neighbours+2):
                if i<lyr.dim:
                    diff = math.fabs(data[k]-lyr.neus[i].value)
                    lyr.neus[i].wt[winner1] += learningConst/l*(1-diff/(l*interval));
                i=i+1
                
            max2 = 0
            for i in range(lyr.dim):
                max2= max(max2,lyr.neus[i].wt[winner1]);
                
                
            for i in range(lyr.dim):
                lyr.neus[i].wt[winner1] = lyr.neus[i].wt[winner1]/max2
                
        iterations=iterations+1
        
        if (math.fabs(e1-le1) <= epsilon) and (math.fabs(e2-le2) <= epsilon):
            print("No. of iterations =", iterations);
            break;
        le1 = e1
        e1 = 0
        le2 = e2
        e2 = 0
     
        
    return nw

In [13]:
def mlvq(nw,data,ndx):
    e=0
    le=0
    iterations=0
    min2=0
    widthFactor=0
    interval=0
    temp=0
    min1=max1=data[0]
    
    for k in range(train_num_line):
        max1=max(max1,data[k])
        min1=min(min1,data[k])
    for j in range(1,nw.num_layers):
        lyr=nw.layers[j]
        for i in range(lyr.dim):
            lyr.neus[i].value=min1+(i+0.5)/(lyr.dim)*(max1-min1)
    
    
    while(1):
        for k in range(train_num_line):
            winner1=findWinner(data[k],nw.layers[1])
            nw.layers[1].neus[winner1].value *= (1-learningConst)
            nw.layers[1].neus[winner1].value += learningConst*data[k];
            e+= math.fabs(data[k]-nw.layers[1].neus[winner1].value)
            
        iterations=iterations+1
        
        if (math.fabs(e-le) <= epsilon):
            print("No. of iterations =", iterations);
            break;
            
        le = e
        e = 0
        
    lyr=nw.layers[1]
    for j in range(lyr.dim):
        if (j==lyr.dim-1):
            widthFactor = math.fabs(lyr.neus[j].value - lyr.neus[j-1].value)/mlvq_lwidth
        elif (j==0):
            widthFactor = math.fabs(lyr.neus[j].value - lyr.neus[j+1].value)/mlvq_lwidth
        else:
            min2=min(math.fabs(lyr.neus[j].value - lyr.neus[j-1].value), math.fabs(lyr.neus[j].value-lyr.neus[j+1].value))
            widthFactor=min2/mlvq_lwidth
        
        interval=(max1-min1)/nw.layers[2].dim
        cur=min1
        
        for i in range(nw.layers[2].dim):
            temp=0-pow(cur-lyr.neus[j].value,2)/widthFactor
            nw.layers[2].neus[i].wt[j]=math.exp(temp)
            cur += interval
            
    return nw

In [14]:
def pfkp(nw,data,ndx):
    e=0
    le=0
    iterations=0
    min2=0
    widthFactor=0
    interval=0
    temp=0
    min1=max1=data[0]
    
    for k in range(train_num_line):
        max1=max(max1,data[k])
        min1=min(min1,data[k])
    for j in range(1,nw.num_layers):
        lyr=nw.layers[j]
        for i in range(lyr.dim):
            lyr.neus[i].value=min1+(i+0.5)/(lyr.dim)*(max1-min1)
    
    alpha=[0]*nw.layers[1].dim
    beta=[0]*nw.layers[1].dim
    sigma=[0]*nw.layers[1].dim
    tau=[0]*nw.layers[1].dim
    phi=[0]*nw.layers[1].dim
    
    
    while(1):
        for k in range(train_num_line):
            winner1=findWinner(data[k],nw.layers[1])
            nw.layers[1].neus[winner1].value *= (1-learningConst)
            nw.layers[1].neus[winner1].value += learningConst*data[k];
            e+= math.fabs(data[k]-nw.layers[1].neus[winner1].value)
            
        iterations=iterations+1
        
        if (math.fabs(e-le) <= epsilon):
            print("No. of iterations =", iterations);
            break;
            
        le = e
        e = 0
        
    le = 0 
    e = 0
    lyr = nw.layers[1]
    for j in range(lyr.dim):
        phi[j] = alpha[j] = beta[j] = sigma[j] = tau[j] = lyr.neus[j].value;
        
    for k in range(train_num_line):
        winner= findWinner(data[k], nw.layers[1])
        phi[winner] *= (1-pfkp_lwidth);
        phi[winner] += pfkp_lwidth*data[k];
        if(winner):
            alpha[winner]=sigma[winner-1]
        else:
            alpha[winner] = min(alpha[winner],data[k])
        beta[winner]= min(beta[winner],phi[winner])
        sigma[winner]= max(sigma[winner],phi[winner])
        if(winner == nw.layers[1].dim - 1):
            tau[winner]= max(tau[winner],data[k])
        else:
            tau[winner] = beta[winner+1]
            
    lyr=nw.layers[2]
    for j in range(nw.layers[1].dim):
        for i in range(lyr.dim):
            if(lyr.neus[i].value <= alpha[j] or lyr.neus[i].value >=tau[j]):
                lyr.neus[i].wt[j]=0
            elif(lyr.neus[i].value < beta[j]):
                slope = 1/(beta[j]-alpha[j])
                lyr.neus[i].wt[j] = slope * (lyr.neus[i].value - alpha[j])
            elif(lyr.neus[i].value >= beta[j] and lyr.neus[i].value <=sigma[j]):
                lyr.neus[i].wt[j]=1
            else: 
                slope = 1/(sigma[j]-tau[j])
                lyr.neus[i].wt[j]= slope * (lyr.neus[i].value - tau[j]) 
            
    return nw

In [15]:
import csv
def printCluster(nw,ndx):
        
    f = open("cluster_dct.txt", "a")
    f.write(" ")
    f.write(str(nw.layers[2].dim))
    f.write("\n ")
    
    fd = open("mf_dct.txt", "a")
    print(nw.layers[2].dim)
    val=[]*nw.layers[2].dim
    for j in range(nw.layers[2].dim):
        val.append(round(nw.layers[2].neus[j].value,6))
        f.write(str(val[j]))
        f.write(" ")
        fd.write(str(val[j]))
        fd.write("\n")
    f.write("\n ")
            
    i=0
    wt=[0]*nw.layers[1].dim*nw.layers[2].dim
    for j in range(nw.layers[1].dim):
        for k in range(nw.layers[2].dim):
            wt[i]=nw.layers[2].neus[k].wt[j]
            f.write(str(round(wt[i],6)))
            f.write(" ")
            fd.write(str(round(wt[i],6)))
            fd.write("\n")
            i=i+1
        f.write("\n ")
    f.close()   
    fd.close()

    return

In [16]:
#return network 
def initNN(ndx):
    x=[]
    nw=network(3)
    
#########################  LAYER 1: INPUT LAYER  #############################
    layer1=layer(1,0,x) #numofneurons, not ipneu, data
    nw.addLayers(0,layer1) #(layernum, layer)
    
#######################  LAYER 2: CENTROID LAYER  #############################
    layer2=layer(clusters,0,x) 
    nw.addLayers(1,layer2)
#   for i in range(1):
    buildConnections(layer1,layer2,0,0,1)
    
#######################  LAYER 3: MEMBERSHIP LAYER  #############################
    layer3=layer(discreteSamplePoints,0,x)
    nw.addLayers(2,layer3) 
    for i in range(clusters):
        buildConnections(layer2,layer3,i,0,1)
    f = open("cluster_dct.txt", "a")
    f.write(col_names[ndx])
    f.close()
    print(col_names[ndx])
#     nw=mlvq(nw, data_train.iloc[:,ndx],ndx)
    nw=dct(nw, data_train.iloc[:,ndx],ndx)
#     nw=pfkp(nw, data_train.iloc[:,ndx],ndx)

    printCluster(nw,ndx)
    return 

In [17]:
def complement(neu):
    return (1-neu.actFunc_arg[0])

In [18]:
def linear(neu):
    return neu.actFunc_arg[0]

In [19]:
#
def classifier(neus):
    classification=0
    min1 = neus.input[0].activation
    
    for i in range(1,neus.num_inputs):
        if (min1>neus.input[i].activation):
            classification=i
            min1= neus.input[i].activation
        
    if (min1<0.5*neus.num_inputs):
        return classification
    else:
        return -1

In [20]:
def deviation(neus):
    dev=0
    
    for i in range(neus.num_inputs):
        dev+= math.fabs(neus.wt[i]-neus.input[i].activation)
        
    return dev

In [21]:
def dotProduct(neus):
    sum=0
    
    for i in range(neus.num_inputs):
        sum += neus.input[i].activation*neus.wt[i]
            
    return sum

In [22]:
def setCombFunc(neus,combfunc):
    neus.combFunc = combfunc 

In [23]:
def setActFunc(neus,actfunc):
    neus.actFunc = actfunc

In [24]:
def initNN1(x):
    nw = network(6)
    nw.num_feature=num_input #input
    nw.num_pattern=num_output #out
    
#########################  LAYER 1: INPUT LAYER  #############################
    
    # create input layer
    lyr=layer(num_input,1,x) #dim,input neu, dataset
    nw.addLayers(0,lyr)
    
#     print("layer 1 with ",nw.layers[0].dim," neurons\n");
#     for j in range(nw.layers[0].dim):
#         print("membership function of \n")
#         print(nw.layers[0].ipneus[j].mf,"\n") 



#######################  LAYER 2: ANTECEDENT LAYER  #############################
    
    # create antecedent layer
    lyr=layer(num_input*num_output,0,x) #dim, not input neu, dataset
    nw.addLayers(1,lyr)
    
    # make connections between antecedent and input layers
    for i in range(nw.num_feature):
        for j in range(nw.num_pattern):
            neu = lyr.neus[i*nw.num_pattern+j]
            neu.ip_type=0
            neu.ipneu_input[0]=nw.layers[0].ipneus[i]
            neu.wt[0]=1.0
            neu.num_inputs=1   
            
    
#######################  LAYER 3: RULE BASE LAYER  #############################
    
    # create clause combination layer
    lyr=layer(num_output,0,x) #dim, not input neu, dataset
    nw.addLayers(2,lyr)
    
    # make connections between clause combination and antecedent layers
    for i in range(lyr.dim):
        total=0
        lyr.neus[i].num_inputs = nw.num_feature
        setCombFunc(lyr.neus[i],0) #DotProduct
        setActFunc(lyr.neus[i],1) #complement
        for j in range(nw.num_feature):
            lyr.neus[i].input[j] = nw.layers[1].neus[j*nw.num_pattern+i]
        for j in range(nw.num_feature):
            lyr.neus[i].wt[j]=1/calcIntersection(nw,i,j);
            #print("i:",i," j:",j)
            total+=lyr.neus[i].wt[j]
        for j in range(nw.num_feature):
            lyr.neus[i].wt[j] /= total
            

#################  LAYER 4: POSSIBILITY DISTRIBUTION LAYER  #############################
    
    # create possibility distribution layer
    lyr = layer(nw.num_pattern*nw.num_pattern,0,x)
    nw.addLayers(3,lyr)
    
    # make connections btwn possibility distribution and caluse combination layers
    for i in range(nw.num_pattern):
        for j in range(nw.num_pattern):
            neu=lyr.neus[i*nw.num_pattern+j]
            setCombFunc(neu,0) #DotProduct
            setActFunc(neu,1) #complement
            neu.input[0]=nw.layers[2].neus[i]
            if (i==j):
                neu.wt[0]=0
            else:
                neu.wt[0]=1
            neu.num_inputs=1
    
    
#################  LAYER 5: OUTPUT DEVIATION LAYER  #############################
 
    # create output deviation layer
    lyr = layer(nw.num_pattern, 0, x)
    nw.addLayers(4,lyr)
    
    # make connections btwn output dev and poss dis layers
    for i in range(lyr.dim):
        lyr.neus[i].num_inputs = nw.num_pattern
        setCombFunc(lyr.neus[i],1) #deviation
        setActFunc(lyr.neus[i],0) #linear
        for j in range(nw.num_pattern):
            lyr.neus[i].input[j] = nw.layers[3].neus[i*nw.num_pattern+j]
            if (i==j):
                lyr.neus[i].wt[j]=1
            else:
                lyr.neus[i].wt[j]=0
    

#################  LAYER 6: OUTPUT DEVIATION LAYER  #############################

    #create final layer
    lyr = layer(1,0,x)
    nw.addLayers(5,lyr)
    
    # make connections btwn final and output deviation layers
    lyr.neus[0].num_inputs=nw.num_pattern
    setCombFunc(lyr.neus[0],2) #classifier
    setActFunc(lyr.neus[0],0) #linear
    for i in range(nw.num_pattern):
        lyr.neus[0].input[i] = nw.layers[4].neus[i]

    
    yagerTest(nw)
    return

In [25]:
#neus.combFunc={{none,-1},{dotProduct,0},{deviation,1},{classifier,2} };
#neus.actFunc={{none,-1},{linear,0}, {complement,1} }

In [26]:
def calcIntersection(nw,cluster, feature):
    intersection=0
    ipneu=nw.layers[0].ipneus[feature]
    
    mf=ipneu.mf
    for i in range(1,nw.num_pattern+1):
        if (cluster != i-1):
            for j in range(ipneu.numPt):
                intersection+=min(mf[cluster+1][j],mf[i][j])

    if (intersection==0):
        return 0.001
    else:
        return intersection

In [27]:
def yagerTest(nw):
    correct=0
    wrong=0
    train_output=[0]*train_num_line
    train_feature = (data_train.iloc[:,[0,1,2,3]]).values
    train_pattern = (data_train.iloc[:,[4]]).values
    test_feature = (data_test.iloc[:,[0,1,2,3]]).values
    test_pattern = (data_test.iloc[:,[4]]).values
    #print(feature)

    for i in range(train_num_line):
        processInput_train(nw,train_feature[i],train_pattern[i],i)
        #mfCluster_y[i]=pattern[i][0]
        train_output[i]=activateNetwork(nw);
        
    f = open("result_fnn_dct.txt", "w")
        
    for i in range(train_num_line):
        if train_output[i] == train_pattern[i][0]:
            print("correct:",train_output[i]," ",train_pattern[i][0])
            f.write("correct: ")
            f.write(str(train_output[i]))
            f.write(" ")
            f.write(str(train_pattern[i][0]))
            f.write("\n")
            correct+=1
        else:
            print("wrong:",train_output[i]," ",train_pattern[i][0]) 
            f.write("wrong: ")
            f.write(str(train_output[i]))
            f.write(" ")
            f.write(str(train_pattern[i][0]))
            f.write("\n")
            wrong+=1
            
    print("Classification rate:",correct/train_num_line)
    f.write("\n")
    f.write("Classification rate: ")
    f.write(str(correct/train_num_line))
    
    f.close()
    
    for i in range(test_num_line):
        processInput_test(nw,test_feature[i],test_pattern[i],i)
    
    return

In [28]:
def activateNetwork(nw):
    for i in range(nw.layers[1].dim):
        neu=nw.layers[1].neus[i]
        neu.activation=neu.ipneu_input[0].value[i%nw.num_pattern]

    for j in range(2,nw.num_layers):
        for i in range(nw.layers[j].dim):
            neu = nw.layers[j].neus[i]
            if(neu.combFunc==0):
                neu.actFunc_arg[0] = dotProduct(neu)
            if(neu.combFunc==1):
                neu.actFunc_arg[0] = deviation(neu)
            if(neu.combFunc==2):
                neu.actFunc_arg[0] = classifier(neu)
                
            if(neu.actFunc==0):
                neu.activation = linear(neu)
            if(neu.actFunc==1):
                neu.activation = complement(neu)
    
    return nw.layers[5].neus[0].activation

In [29]:
def processInput_train(nw,x,y,count):
    
    for i in range(nw.num_feature):
        ipneu=nw.layers[0].ipneus[i]
        ipneu.ip_value=x[i]
        mf=ipneu.mf
        ##find the disagreement score for input value
        winner=findWinner1(x[i],mf[0],ipneu.numPt)
        for j in range(clusters):
            ipneu.value[j] = 1-mf[j+1][winner]
            fuzzytrain_x[count][(i*clusters)+j]=mf[j+1][winner]#ipneu.value[j]
        
        ##find the similarity score for output value             
        winner=findWinner1(y[0],outmf[0],ipneu.numPt)
        for j in range(clusters):
            fuzzytrain_y[count][j] = outmf[j+1][winner]

In [30]:
def processInput_test(nw,x,y,count):
    
    for i in range(nw.num_feature):
        ipneu=nw.layers[0].ipneus[i]
        ipneu.ip_value=x[i]
        mf=ipneu.mf
        ##find the disagreement score for input value
        winner=findWinner1(x[i],mf[0],ipneu.numPt)
        for j in range(clusters):
            ipneu.value[j] = 1-mf[j+1][winner]
            fuzzytest_x[count][(i*clusters)+j]=mf[j+1][winner]#ipneu.value[j]
        
        ##find the similarity score for output value             
        winner=findWinner1(y[0],outmf[0],ipneu.numPt)
        for j in range(clusters):
            fuzzytest_y[count][j] = outmf[j+1][winner]

In [31]:
num_var=num_input+num_output
x=[]
f = open("cluster_dct.txt", "w")
f.write(str(num_input))
f.write(str(num_output))
f.write("\n")
f.close()

fd = open("mf_dct.txt", "w")
fd.close()

for i in range(num_input+1):
    initNN(i)

with open('mf_dct.txt', 'r') as f:
    i = f.read().split()
    for elem in i:
        try:
            x.append(float(elem))
        except ValueError:
            pass
print(len(x))

outmf=[[0 for i in range(discreteSamplePoints)] for j in range(clusters+1)]    
fuzzytrain_x = [[0 for i in range((num_input*clusters))] for j in range(train_num_line)]   
fuzzytrain_y = [[0 for i in range((clusters))] for j in range(train_num_line)]
fuzzytest_x = [[0 for i in range((num_input*clusters))] for j in range(test_num_line)]   
fuzzytest_y = [[0 for i in range((clusters))] for j in range(test_num_line)] 
initNN1(x)

SepalLength
No. of iterations = 13
50
SepalWidth
No. of iterations = 33
50
PetalLength
No. of iterations = 9
50
PetalWidth
No. of iterations = 18
50
TYPE


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in double_scalars


ZeroDivisionError: division by zero

In [ ]:
import numpy
fuzzytrain_x = numpy.array(fuzzytrain_x)
fuzzytrain_y = numpy.array(fuzzytrain_y)

print("Shape of X",fuzzytrain_x.shape)
print("Shape of y",fuzzytrain_y.shape)
print("Examples of X\n",fuzzytrain_x[:3])
print("Examples of y\n",fuzzytrain_y[:3])

In [ ]:
fuzzytest_x = numpy.array(fuzzytest_x)
fuzzytest_y = numpy.array(fuzzytest_y)

print("Shape of X",fuzzytest_x.shape)
print("Shape of y",fuzzytest_y.shape)
print("Examples of X\n",fuzzytest_x[:3])
print("Examples of y\n",fuzzytest_y[:3])

In [ ]:
# from keras.utils import np_utils
# mfCluster_y=np_utils.to_categorical(mfCluster_y,num_classes=3)
print("Shape of y_train",fuzzytrain_y.shape)

In [ ]:
def mlp(x):
    global fuzzytrain_x
    global fuzzytrain_y
    
    mlp=Sequential()
    print('x[0]:',x[0],' x[1]:',x[1],' x[2]:',x[2])
    f = open("GA_3layers_200_100_200.txt", "a")
    f.write("x[0]:")
    f.write(str(x[0]))
    f.write("x[1]:")
    f.write(str(x[1]))
    f.write("x[2]:")
    f.write(str(x[2]))
    f.write("\n")
    
    mlp.add(Dense(x[0],input_dim=num_input*clusters,activation='relu'))
    mlp.add(Dense(x[1],activation='relu'))
    mlp.add(Dense(x[2],activation='relu'))
    mlp.add(Dropout(0.2))
    mlp.add(Dense(clusters,activation='softmax'))
    mlp.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

    #mlp.fit(mfCluster_x,mfCluster_y,validation_data=(mfCluster_x,mfCluster_y),batch_size=20,epochs=20,verbose=1)
    mlp.fit(fuzzytrain_x,fuzzytrain_y,validation_data=(fuzzytest_x,fuzzytest_y),batch_size=20,epochs=70,verbose=1)

    prediction=mlp.predict(fuzzytest_x)
    length=len(prediction)
    actual=np.argmax(fuzzytest_y,axis=1)
    predict_label=np.argmax(prediction,axis=1)

    accuracy=np.sum(actual==predict_label)/length * 100 
    f.write("Accuracy of the dataset: ")
    f.write(str(accuracy))
    f.write("\n\n")
    print("Accuracy of the dataset: ",accuracy,"\n")
    
    return -accuracy

In [ ]:
# import numpy as np
# from geneticalgorithm import geneticalgorithm as ga
# f = open("GA_3layers_200_100_200.txt", "w")
# varbound=np.array([[1,200],[1,100],[1,200]])
# vartype=np.array([['int'],['int'],['int']])

# model=ga(function=mlp,dimension=3,variable_type_mixed=vartype,variable_boundaries=varbound)

# model.run()

In [ ]:
mlp=Sequential()
mlp.add(Dense(500,input_dim=num_input*clusters,activation='elu'))
# mlp.add(Dense(200,activation='relu'))
# mlp.add(Dense(200,activation='relu'))
mlp.add(Dropout(0.2))
mlp.add(Dense(clusters,activation='softmax'))
mlp.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
mlp.summary()

In [ ]:
mlp.fit(fuzzytrain_x,fuzzytrain_y,validation_data=(fuzzytest_x,fuzzytest_y),batch_size=20,epochs=20,verbose=1)

prediction=mlp.predict(fuzzytest_x)
length=len(prediction)
actual=np.argmax(fuzzytest_y,axis=1)
predict_label=np.argmax(prediction,axis=1)

accuracy=np.sum(actual==predict_label)/length * 100 
print("Accuracy of the dataset",accuracy,"\n")

In [ ]:
correct=0
wrong=0
f = open("result_fmlp_dct.txt", "w")
for i in range(len(predict_label)):
    if predict_label[i] == actual[i]:
        print("correct:",predict_label[i]," ",actual[i])
        f.write("correct: ")
        f.write(str(predict_label[i]))
        f.write(" ")
        f.write(str(actual[i]))
        f.write("\n")
        correct+=1
    else:
        print("wrong:",predict_label[i]," ",actual[i]) 
        f.write("wrong: ")
        f.write(str(predict_label[i]))
        f.write(" ")
        f.write(str(actual[i]))
        f.write("\n")
        wrong+=1
            
print("Classification rate:",correct/len(predict_label))
f.write("\n")
f.write("Classification rate: ")
f.write(str(correct/len(predict_label)))

f.close()